In [14]:
import pandas as pd
import geopandas as gpd
import json

In [15]:
db = pd.read_csv("..\database.csv")
rydde = db[931:17798]

In [16]:
gdf = gpd.GeoDataFrame(rydde, geometry=gpd.points_from_xy(rydde.longitude, rydde.latitude))

In [17]:
gdf.date = pd.to_datetime(gdf.date)

In [26]:
#Svinøya 30x30
lat_min = 62.345252708439276
lat_max = 62.61474729156072
lng_min = 5.978375939307136
lng_max = 6.561624060692863
cell_rows, cell_columns = 16, 16

In [19]:
def generateCells(lat_min, lng_min, lat_max, lng_max, cell_rows, cell_columns):
    half_cell_height = (lat_max - lat_min) / (2 * (cell_rows - 1))
    half_cell_width = (lng_max - lng_min) / (2 * (cell_columns - 1))
    lat_point = lat_min
    row_count = 0
    cells = []
    while (row_count < cell_rows):
        lng_point = lng_min
        column_count = 0
        while (column_count < cell_columns):
            lat_0 = lat_point - half_cell_height
            lng_0 = lng_point - half_cell_width
            lat_1 = lat_point + half_cell_height
            lng_1 = lng_point + half_cell_width
            cells.append({"lng_min":lng_0, "lat_min": lat_0, "lat_max":lat_1, "lng_max":lng_1})
            lng_point += 2 * half_cell_width
            column_count+=1
        lat_point += 2 * half_cell_height
        row_count+=1
    return cells

In [20]:
cells = generateCells(lat_min, lng_min, lat_max, lng_max, cell_rows, cell_columns)

In [21]:
pd.Series(json.dumps(cells)).to_csv("..\cells.csv")

In [22]:
def getCellData(cell):
        data = gdf.cx[cell["lng_min"]: cell["lng_max"], cell["lat_min"] : cell["lat_max"]]
        return data

In [23]:
def createBoundsDf(cells):
    i = 0;
    udf = pd.DataFrame(columns=["date", "boundsId", "weight", "quantity"])
    for cell in cells:
        data = getCellData(cell)
        newdf = pd.DataFrame(data)
        newdf["boundsId"] = i
        newdf = newdf[["date", "boundsId", "weight", "quantity"]]
        udf = udf.append(newdf, ignore_index=True)
        i+=1
    return udf

In [24]:
bdf = createBoundsDf(cells)

In [25]:
bdf.to_csv("..\cells_database.csv")